In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import gc
import torch
import zipfile

In [3]:
#archivo = zipfile.ZipFile('ml10m.zip')
archivo = zipfile.ZipFile('ml-1m.zip')

In [4]:
archivo.extractall('')

In [8]:
movie_col = ['movie_id', 'titulo', 'genero']
movie = pd.read_csv('ml-1m/movies.dat', sep='::', names=movie_col)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [9]:
movie.head()

,movie_id,titulo,genero
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
rating_col = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=rating_col, usecols=range(3))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [11]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [12]:
usuario_col = ['user_id']
usuarios = pd.read_csv('ml-1m/users.dat', sep='::', names=usuario_col, usecols=range(1))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [13]:
usuarios.head()

,user_id
0,1
1,2
2,3
3,4
4,5


In [14]:
movie_rating = pd.merge(movie, ratings)

In [15]:
movie_rating.head()

,movie_id,titulo,genero,user_id,rating
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5


In [16]:
movie_rating.shape

(1000209, 5)

In [17]:
datos_completos = movie_rating.pivot_table(index=['user_id'],columns=['titulo'],values='rating')

In [18]:
datos_completos.head()

titulo,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
datos_completos.shape

(6040, 3706)

In [20]:
datos_norm = datos_completos.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [21]:
datos_norm.fillna(0, inplace=True)

In [22]:
datos_norm.head()

titulo,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
gc.collect()

1480

In [24]:
matrix = datos_norm.values

In [25]:
matrix.shape

(6040, 3706)

In [26]:
movie_similar = cosine_similarity(matrix.T)

In [27]:
movie_sim_df = pd.DataFrame(movie_similar, index = datos_norm.T.index, columns = datos_norm.T.index)

In [28]:
movie_sim_df.head()

titulo,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kj�rlighetens kj�tere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
titulo,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck (1971)",1.000000,0.025799,0.016365,0.018348,-0.019751,0.00000,-0.037170,0.064196,-0.011439,-0.105361,...,0.001049,0.003048,0.000000,-0.012493,0.0,0.006963,-1.799323e-02,0.000000,0.044986,-0.018606
'Night Mother (1986),0.025799,1.000000,-0.024565,-0.016362,-0.011022,0.00000,0.010923,-0.001025,-0.022830,-0.005153,...,0.015387,-0.016177,-0.038464,-0.012244,0.0,-0.122795,-3.809734e-07,0.000000,0.000000,-0.000915
'Til There Was You (1997),0.016365,-0.024565,1.000000,0.103186,0.000430,0.03037,-0.026358,0.011110,0.033666,-0.037697,...,-0.009768,0.017585,0.000000,0.050740,0.0,0.006322,-3.558901e-02,0.000000,0.002490,0.006594
"'burbs, The (1989)",0.018348,-0.016362,0.103186,1.000000,-0.053175,0.00000,0.010785,-0.032967,0.039810,-0.099320,...,0.019417,0.019554,0.014485,0.016478,0.0,-0.005922,-4.999707e-02,0.000000,-0.009842,0.004350
...And Justice for All (1979),-0.019751,-0.011022,0.000430,-0.053175,1.000000,0.00000,0.002268,0.007789,-0.047307,0.068875,...,-0.056230,-0.006121,0.002806,-0.002000,0.0,-0.104634,1.076627e-02,-0.028115,0.000000,-0.029400


In [29]:
gc.collect()

132

In [30]:
torch.cuda.empty_cache()

In [31]:
usuarios_similar = cosine_similarity(matrix)

In [32]:
usuarios_sim_df = pd.DataFrame(usuarios_similar, index = datos_norm.index, columns = datos_norm.index)

In [33]:
usuarios_sim_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.032665,-0.032267,0.016359,-0.016774,0.045229,0.014314,0.013465,0.061905,-0.004469,...,0.012852,0.034246,0.054216,-0.003729,0.069408,-0.038463,0.000283,0.000000,0.011725,0.005210
2,0.032665,1.000000,0.023592,-0.016699,-0.032420,-0.019154,0.062835,-0.002045,0.060978,-0.004654,...,-0.002849,0.012514,0.080611,-0.006570,0.076127,0.088853,0.054506,-0.013096,0.032720,-0.006562
3,-0.032267,0.023592,1.000000,0.042521,-0.031153,-0.010965,0.057895,-0.040328,-0.021620,0.003036,...,-0.016524,0.005764,-0.027753,0.000000,0.009517,0.028658,0.001249,0.046408,0.048975,-0.041902
4,0.016359,-0.016699,0.042521,1.000000,-0.002730,0.032028,-0.031510,0.031002,0.021538,-0.038158,...,-0.038547,0.002558,-0.004579,0.000000,-0.051202,0.043721,0.014981,-0.073011,-0.011580,0.035714
5,-0.016774,-0.032420,-0.031153,-0.002730,1.000000,-0.045909,0.009393,0.045783,0.015280,-0.004929,...,0.019115,0.013596,0.000476,0.011270,0.061089,0.049306,-0.023689,-0.035286,0.017825,0.064367


In [34]:
def top_movies(movie):
    movie_sim = movie_sim_df.sort_values(by = movie, ascending=False).index[1:6]
    valores_sim = movie_sim_df.sort_values(by = movie, ascending=False).loc[:,movie].tolist()[1:6]
    zipped = zip(movie_sim, valores_sim,)
    print('Películas similares a ', movie)
    for mov, sim in zipped:
        print(mov, 'tiene similitud de ', sim)

In [43]:
def top_users(usuario):
    
    if usuario not in datos_norm.index:
        return('No existe usuario')
    
    else:
        valores_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).loc[:,usuario].tolist()[1:6]
        usuario_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).index[1:6]
        zipped = zip(usuario_sim, valores_sim,)
        for user, sim in zipped:
            print('El usuario ', user, 'tiene similitud de ', sim) 

In [54]:
#Predice la calificación que cierto usuario le daría a alguna película

def pred_rating(movie, usuario):
    valores_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).loc[:,usuario].tolist()[1:1000]
    usuario_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).index[1:1000]
    ratings = []
    pesos = []
    
    for i, j in enumerate(usuario_sim):
        rating = datos_completos.loc[j, movie]
        similares = usuario_sim[i]
        if np.isnan(rating):
            continue
        elif not np.isnan(rating):
            ratings.append(rating*similares)
            pesos.append(similares)
    return sum(ratings)/sum(pesos)

In [44]:
top_movies('10 Things I Hate About You (1999)')

Películas similares a  10 Things I Hate About You (1999)
Cruel Intentions (1999) tiene similitud de  0.12236272859399336
Can't Hardly Wait (1998) tiene similitud de  0.11259278521705773
Never Been Kissed (1999) tiene similitud de  0.10475264994330213
She's All That (1999) tiene similitud de  0.10353135275953183
Varsity Blues (1999) tiene similitud de  0.10343753833686739


In [47]:
top_movies('Toy Story (1995)')

Películas similares a  Toy Story (1995)
Toy Story 2 (1999) tiene similitud de  0.4858766907292213
Raiders of the Lost Ark (1981) tiene similitud de  0.2936132807321098
Bug's Life, A (1998) tiene similitud de  0.2869373756254773
Shawshank Redemption, The (1994) tiene similitud de  0.27135995322691914
Back to the Future (1985) tiene similitud de  0.26796936852196224


In [51]:
top_users(2010)

El usuario  302 tiene similitud de  0.3726655225017733
El usuario  5367 tiene similitud de  0.35166312979052566
El usuario  1613 tiene similitud de  0.3363071195188399
El usuario  710 tiene similitud de  0.3352453933867504
El usuario  3389 tiene similitud de  0.33402927659156917


In [57]:
pred_rating('Toy Story (1995)', 15)

4.089547700727573